In [2]:
import cv2
import numpy as np

In [3]:
# Canny Edge Detection

def canny(img):
    if img is None:
        cap.release()
        cv2.destroyAllWindows()
        exit()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), cv2.BORDER_DEFAULT)
    canny_img = cv2.Canny(blur, 50, 150)
    return(canny_img)

def region_of_interest(canny_img):
    height = canny_img.shape[0]
    width = canny_img.shape[1]
    triangle = np.array([[
        (200,height),
        (width,height),
        (800,420)
    ]])
    mask = np.zeros_like(canny_img)
    cv2.fillPoly(mask, triangle, (255))
    masked_image = cv2.bitwise_and(canny_img, mask)
    return(masked_image)

def houghLines(masked_image):
    lines = cv2.HoughLinesP(masked_image, 2, np.pi/180, 100, np.array([]), minLineLength = 75, maxLineGap = 5)
    return(lines)

def display_lines(masked_image, lines):
    line_image = np.zeros_like(masked_image)
    line_image = cv2.cvtColor(line_image, cv2.COLOR_GRAY2RGB)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(line_image, (x1, y1), (x2, y2), (0,255,255), 10)
    return(line_image)           

def final_result(img,line_image):
    final_image = cv2.addWeighted(img, 0.8, line_image, 2, 1)
    return(final_image)


In [4]:
cap = cv2.VideoCapture("test1_video.mp4")

while cap.isOpened():
    _, frame = cap.read()
    canny_image = canny(frame)
    roi_image = region_of_interest(canny_image)
    
    lines = houghLines(roi_image)
    line_image = display_lines(roi_image, lines)
    combo_image = final_result(frame, line_image)
    cv2.imshow("",combo_image)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()